# Create 7777 video - 'everyday' for NFT

## Prepare everything

In [ ]:
import os
import shutil
import cv2
from skimage import io
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches

from tqdm.notebook import tqdm

# Setup video parameters

In [ ]:
# Get all filenames
imgs = sorted(glob('img_aligned/20*'))

# Extract number of images
N_total = len(imgs)
N_total

In [ ]:
# How many images to smooth at once
smooth = 60

In [ ]:
# How many days to jump at every image
step_size = 1

In [ ]:
# Get start indeces for images
ids = [i*step_size for i in range((N_total+smooth)//step_size+1)]
len(ids)

In [ ]:
# Specify frames per second
fps = 60

print('Video length: %.2f seconds.' % (len(ids)/fps))

# Rescale all photos

In [ ]:
# Or with command line
#for i in `ls img_aligned/*`; do convert $i -resize 2560x1440 img_wqhd/${i:12}; done

In [ ]:
# Resolution information
ultraHD = (3840, 2160)
WQHD = (2560, 1440)
fullHD = (1920, 1080)

# Create video

In [ ]:
# Get all filenames of the small images
imgs_small = sorted(glob('img_small/20*'))

### Mean image of 1080p size

In [ ]:
# Save images to disk
out_dir = 'img_video_mean'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# To keep track what was already loaded
already_loaded = []

for i in tqdm(ids):
    
    # Collect indeces of images
    imgs_idx = np.arange(np.clip(i-smooth, 0, N_total-1), np.clip(i, 0, N_total-1)+1)

    # Collect images relevant for the group
    group_names = np.array(imgs_small)[imgs_idx]
    
    # Detect which one is new to load
    new_to_load = np.setdiff1d(group_names, already_loaded)
    
    if len(new_to_load)==0:
        pass
    elif i==0:
        imgs_group = np.array([io.imread(f) for f in new_to_load])
    else:
        img_new = np.array([io.imread(f) for f in new_to_load])
        imgs_group = np.vstack((imgs_group, img_new))
        
    # Cut imgs_group to write size
    n_offset = (i - N_total)
    if n_offset <= 0:
        n_offset = 0
    elif n_offset%2==0:
        n_offset -= 1
    imgs_group = imgs_group[-smooth+n_offset:]
    
    # Create composition image
    img_comp = np.mean(imgs_group, axis=0).astype('int')
    
    # Create out_filename
    out_filename = os.path.join(out_dir, '%04d.jpg' % (i + 1))
    
    # Save composition image
    io.imsave(out_filename, img_comp.astype('uint8'))

    # Keep track of what has already been loaded
    already_loaded = group_names

In [ ]:
# Use either code (the one that works) to create the video
!cat img_video_mean/*jpg | ffmpeg -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 video_mean_60fps.mp4
#!ffmpeg -r 30 -f image2 -pattern_type glob -i 'img_video_mean/*.jpg' -c:v libx264 -profile:v high video_mean.mp4